In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    FFF_Train_Decile1 = pd.read_pickle("FFF_Train_Decile1.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile1.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile1, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 1')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 3s 858us/step - loss: 0.5640 - acc: 0.7142 - val_loss: 0.5272 - val_acc: 0.7160
Epoch 35/75
3055/3055 [==============================] - 3s 916us/step - loss: 0.5573 - acc: 0.7208 - val_loss: 0.5271 - val_acc: 0.7160
Epoch 36/75
3055/3055 [==============================] - 3s 962us/step - loss: 0.5598 - acc: 0.7133 - val_loss: 0.5271 - val_acc: 0.7173
Epoch 37/75
3055/3055 [==============================] - 3s 906us/step - loss: 0.5541 - acc: 0.7172 - val_loss: 0.5271 - val_acc: 0.7173
Epoch 38/75
3055/3055 [==============================] - 3s 959us/step - loss: 0.5635 - acc: 0.7087 - val_loss: 0.5270 - val_acc: 0.7160
Epoch 39/75
3055/3055 [==============================] - 3s 922us/step - loss: 0.5592 - acc: 0.7182 - val_loss: 0.5271 - val_acc: 0.7160
Epoch 40/75
3055/3055 [==============================] - 3s 900us/step - loss: 0.5627 - acc: 0.7162 - val_loss: 0.5270 - val_acc: 0.7160
Epoch 41/75
3055/3055 [==============================

3055/3055 [==============================] - 3s 942us/step - loss: 0.5516 - acc: 0.7234 - val_loss: 0.5155 - val_acc: 0.7435
Epoch 19/75
3055/3055 [==============================] - 3s 976us/step - loss: 0.5435 - acc: 0.7329 - val_loss: 0.5182 - val_acc: 0.7395
Epoch 20/75
3055/3055 [==============================] - 3s 967us/step - loss: 0.5465 - acc: 0.7381 - val_loss: 0.5155 - val_acc: 0.7448
Epoch 21/75
3055/3055 [==============================] - 3s 999us/step - loss: 0.5421 - acc: 0.7270 - val_loss: 0.5159 - val_acc: 0.7395

Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 22/75
3055/3055 [==============================] - 2s 680us/step - loss: 0.5440 - acc: 0.7296 - val_loss: 0.5153 - val_acc: 0.7474
Epoch 23/75
3055/3055 [==============================] - 3s 897us/step - loss: 0.5449 - acc: 0.7224 - val_loss: 0.5150 - val_acc: 0.7487
Epoch 24/75
3055/3055 [==============================] - 3s 952us/step - loss: 0.5372 - acc: 0.7277 - val_lo

3055/3055 [==============================] - 3s 911us/step - loss: 0.5481 - acc: 0.7286 - val_loss: 0.5159 - val_acc: 0.7474
Test accuracy: 0.747382199889
Train on 3055 samples, validate on 764 samples
Epoch 1/100
3055/3055 [==============================] - 7s 2ms/step - loss: 0.6499 - acc: 0.6101 - val_loss: 0.5820 - val_acc: 0.7173
Epoch 2/100
3055/3055 [==============================] - 3s 950us/step - loss: 0.5900 - acc: 0.7047 - val_loss: 0.5492 - val_acc: 0.7173
Epoch 3/100
3055/3055 [==============================] - 3s 940us/step - loss: 0.5730 - acc: 0.7070 - val_loss: 0.5393 - val_acc: 0.7186
Epoch 4/100
3055/3055 [==============================] - 3s 922us/step - loss: 0.5661 - acc: 0.7103 - val_loss: 0.5365 - val_acc: 0.7225
Epoch 5/100
3055/3055 [==============================] - 3s 909us/step - loss: 0.5674 - acc: 0.7139 - val_loss: 0.5378 - val_acc: 0.7160
Epoch 6/100
3055/3055 [==============================] - 3s 1ms/step - loss: 0.5611 - acc: 0.7133 - val_loss: 0.532

Epoch 58/100
3055/3055 [==============================] - 3s 901us/step - loss: 0.5456 - acc: 0.7345 - val_loss: 0.5185 - val_acc: 0.7461
Epoch 59/100
3055/3055 [==============================] - 3s 892us/step - loss: 0.5409 - acc: 0.7277 - val_loss: 0.5184 - val_acc: 0.7461
Epoch 60/100
3055/3055 [==============================] - 3s 880us/step - loss: 0.5426 - acc: 0.7290 - val_loss: 0.5183 - val_acc: 0.7461
Epoch 61/100
3055/3055 [==============================] - 3s 979us/step - loss: 0.5457 - acc: 0.7283 - val_loss: 0.5182 - val_acc: 0.7448
Epoch 62/100
3055/3055 [==============================] - 2s 770us/step - loss: 0.5452 - acc: 0.7260 - val_loss: 0.5185 - val_acc: 0.7461
Epoch 63/100
3055/3055 [==============================] - 2s 748us/step - loss: 0.5475 - acc: 0.7313 - val_loss: 0.5184 - val_acc: 0.7461
Epoch 64/100
3055/3055 [==============================] - 3s 881us/step - loss: 0.5441 - acc: 0.7205 - val_loss: 0.5182 - val_acc: 0.7461
Epoch 65/100
3055/3055 [==========

...

Epoch 18/125
3055/3055 [==============================] - 3s 917us/step - loss: 0.5655 - acc: 0.7142 - val_loss: 0.5297 - val_acc: 0.7264
Epoch 19/125
3055/3055 [==============================] - 3s 943us/step - loss: 0.5635 - acc: 0.7113 - val_loss: 0.5291 - val_acc: 0.7251
Epoch 20/125
3055/3055 [==============================] - 3s 984us/step - loss: 0.5653 - acc: 0.7234 - val_loss: 0.5292 - val_acc: 0.7251
Epoch 21/125
3055/3055 [==============================] - 3s 990us/step - loss: 0.5619 - acc: 0.7227 - val_loss: 0.5294 - val_acc: 0.7291
Epoch 22/125
3055/3055 [==============================] - 3s 1ms/step - loss: 0.5634 - acc: 0.7250 - val_loss: 0.5292 - val_acc: 0.7251
Epoch 23/125
3055/3055 [==============================] - 3s 1ms/step - loss: 0.5610 - acc: 0.7155 - val_loss: 0.5284 - val_acc: 0.7251
Epoch 24/125
3055/3055 [==============================] - 3s 949us/step - loss: 0.5618 - acc: 0.7165 - val_loss: 0.5289 - val_acc: 0.7291

Epoch 00024: ReduceLROnPlateau reduci

3055/3055 [==============================] - 3s 989us/step - loss: 0.5596 - acc: 0.7260 - val_loss: 0.5276 - val_acc: 0.7251
Epoch 78/125
3055/3055 [==============================] - 3s 931us/step - loss: 0.5596 - acc: 0.7169 - val_loss: 0.5275 - val_acc: 0.7238
Epoch 79/125
3055/3055 [==============================] - 3s 983us/step - loss: 0.5589 - acc: 0.7110 - val_loss: 0.5274 - val_acc: 0.7238
Epoch 80/125
3055/3055 [==============================] - 3s 1ms/step - loss: 0.5570 - acc: 0.7201 - val_loss: 0.5275 - val_acc: 0.7251
Epoch 81/125
3055/3055 [==============================] - 3s 1ms/step - loss: 0.5595 - acc: 0.7152 - val_loss: 0.5275 - val_acc: 0.7264
Epoch 82/125
3055/3055 [==============================] - 3s 1ms/step - loss: 0.5562 - acc: 0.7175 - val_loss: 0.5274 - val_acc: 0.7251
Epoch 83/125
3055/3055 [==============================] - 3s 1ms/step - loss: 0.5618 - acc: 0.7149 - val_loss: 0.5274 - val_acc: 0.7238
Epoch 84/125
3055/3055 [==============================]

3055/3055 [==============================] - 2s 738us/step - loss: 0.5498 - acc: 0.7172 - val_loss: 0.5201 - val_acc: 0.7356
Epoch 12/100
3055/3055 [==============================] - 3s 1ms/step - loss: 0.5502 - acc: 0.7277 - val_loss: 0.5167 - val_acc: 0.7395
Epoch 13/100
3055/3055 [==============================] - 2s 723us/step - loss: 0.5498 - acc: 0.7254 - val_loss: 0.5153 - val_acc: 0.7474
Epoch 14/100
3055/3055 [==============================] - 3s 1ms/step - loss: 0.5466 - acc: 0.7286 - val_loss: 0.5144 - val_acc: 0.7474
Epoch 15/100
3055/3055 [==============================] - 3s 1ms/step - loss: 0.5459 - acc: 0.7309 - val_loss: 0.5147 - val_acc: 0.7448
Epoch 16/100
3055/3055 [==============================] - 3s 996us/step - loss: 0.5436 - acc: 0.7293 - val_loss: 0.5162 - val_acc: 0.7421
Epoch 17/100
3055/3055 [==============================] - 3s 991us/step - loss: 0.5424 - acc: 0.7316 - val_loss: 0.5136 - val_acc: 0.7474
Epoch 18/100
3055/3055 [=============================

3055/3055 [==============================] - 2s 719us/step - loss: 0.5413 - acc: 0.7332 - val_loss: 0.5127 - val_acc: 0.7461
Epoch 70/100
3055/3055 [==============================] - 2s 673us/step - loss: 0.5399 - acc: 0.7316 - val_loss: 0.5128 - val_acc: 0.7461
Epoch 71/100
3055/3055 [==============================] - 2s 672us/step - loss: 0.5423 - acc: 0.7336 - val_loss: 0.5130 - val_acc: 0.7461
Epoch 72/100
3055/3055 [==============================] - 2s 702us/step - loss: 0.5439 - acc: 0.7273 - val_loss: 0.5129 - val_acc: 0.7461
Epoch 73/100
3055/3055 [==============================] - 2s 722us/step - loss: 0.5347 - acc: 0.7273 - val_loss: 0.5127 - val_acc: 0.7461
Epoch 74/100
3055/3055 [==============================] - 2s 693us/step - loss: 0.5434 - acc: 0.7280 - val_loss: 0.5127 - val_acc: 0.7461
Epoch 75/100
3055/3055 [==============================] - 2s 685us/step - loss: 0.5349 - acc: 0.7293 - val_loss: 0.5128 - val_acc: 0.7461
Epoch 76/100
3055/3055 [=======================

3055/3055 [==============================] - 2s 697us/step - loss: 0.5500 - acc: 0.7257 - val_loss: 0.5163 - val_acc: 0.7500
Epoch 29/50
3055/3055 [==============================] - 2s 692us/step - loss: 0.5491 - acc: 0.7332 - val_loss: 0.5161 - val_acc: 0.7500
Epoch 30/50
3055/3055 [==============================] - 2s 693us/step - loss: 0.5513 - acc: 0.7319 - val_loss: 0.5163 - val_acc: 0.7500
Epoch 31/50
3055/3055 [==============================] - 2s 697us/step - loss: 0.5445 - acc: 0.7290 - val_loss: 0.5160 - val_acc: 0.7500
Epoch 32/50
3055/3055 [==============================] - 2s 695us/step - loss: 0.5481 - acc: 0.7339 - val_loss: 0.5164 - val_acc: 0.7461

Epoch 00032: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 33/50
3055/3055 [==============================] - 2s 681us/step - loss: 0.5453 - acc: 0.7316 - val_loss: 0.5163 - val_acc: 0.7461
Epoch 34/50
3055/3055 [==============================] - 2s 672us/step - loss: 0.5483 - acc: 0.7257 - val_loss: 0.5162 - val_

In [3]:
0.93484

0.93484